In [1]:
import vectorized_sdr as vect
import numpy as np
import scipy 
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import cvxpy as cp

np.random.seed(0)

In [37]:
N_omega = 5
omega = np.linspace(-2, 2, N_omega)
N_z = 15
z = np.linspace(0, 10**-1, N_z)
n = 0.004
rand_pump = np.random.random(2*N_omega - 1)
beta = rand_pump.copy()
vec_to_hankel = vect.omega_vec_to_omega_dof(N_omega)
green_fs = vect.get_green_f(omega,z)

In [38]:
projections = []
for i in range(N_omega):
    for j in range(N_omega):
        proj = np.zeros((N_omega, N_omega))
        proj[i,j] = 1
        projections.append(proj.conj().T)

In [39]:
# Get the dynamics constraints matrices
real_plus_mats_list = []
imag_plus_mats_list = []
real_minus_mats_list = []
imag_minus_mats_list = []
J_def_mat = []
for i in range(len(projections)):
    real_plus_mats, imag_plus_mats, real_minus_mats, imag_minus_mats = vect.dynamics_mat(omega, z, projections[i])
    real_plus_mats_list += real_plus_mats
    imag_plus_mats_list += imag_plus_mats
    real_minus_mats_list += real_minus_mats
    imag_minus_mats_list += imag_minus_mats
    J_def_mat.append(vect.J_def_sdp_mat(N_omega, N_z, projections[i]))

In [52]:
# Verification that the dynamics work properly
beta_vec = np.exp(-(np.linspace(omega[0], omega[-1], 2*N_omega - 1)**2)/0.2)#.random.random(2*N_omega - 1)
beta = vect.vec_to_mat(vect.omega_vec_to_omega_dof(N_omega)@beta_vec)
delta_k = np.diag(1.j*omega)
W_plus = [scipy.linalg.expm((delta_k + beta)*z[i]) for i in range(N_z)]
W_minus = [scipy.linalg.expm((delta_k - beta)*z[i]) for i in range(N_z)]
J = 0.25*(W_plus[-1]@W_plus[-1].conj().T + W_minus[-1]@W_minus[-1].conj().T - 2*np.eye(N_omega))
n = np.trace(J)
vect_W_plus = [vect.mat_to_vec(W_plus[i]) for i in range(N_z)]
vect_W_minus = [vect.mat_to_vec(W_minus[i]) for i in range(N_z)]
vec_W_plus_concat = sparse.vstack(vect_W_plus)
vec_W_minus_concat = sparse.vstack(vect_W_minus)
vec_J = vect.mat_to_vec(J)
x = sparse.vstack([vec_J, vec_W_plus_concat, vec_W_minus_concat, sparse.csc_matrix(beta_vec).T])
y = sparse.vstack([x, 1])

In [53]:
np.trace(J)**2 - np.trace(J.conj().T@J)

(0.002281059572811514+0j)

In [55]:
# Symplective constraints
real_symplectic_constr = []
imag_symplectic_constr = []
for i in range(len(projections)):
    real_symplectic, imag_symplectic = vect.quad_symplectic(N_omega, N_z, projections[i])
    real_symplectic_constr += real_symplectic
    imag_symplectic_constr += imag_symplectic

In [56]:
# Test symplectic constraints work
rand_y = np.hstack([np.random.random((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1), np.eye(1)[0]])
test_symplectic = [rand_y.conj().T@real_symplectic_constr[i*N_z]@rand_y for i in range(len(projections))]
mat_W_plus = vect.vec_to_mat(rand_y[N_omega**2:2*N_omega**2])
mat_W_minus = vect.vec_to_mat(rand_y[(1 + N_z)*(N_omega**2):(2 + N_z)*(N_omega**2)])
true_symplectic = [0.5*np.trace(projections[i]@mat_W_minus@mat_W_plus.conj().T + (projections[i]@mat_W_minus@mat_W_plus.conj().T).conj().T) - np.trace(projections[i]) for i in range(len(projections))]
np.allclose(true_symplectic, test_symplectic)

True

In [57]:
# Various single constraints (limitation on variance of pump, constraint on structure of SDR to be 1, objective function to be non-negative, photon number)
obj_f_mat = vect.obj_f_sdp_mat(N_omega, N_z, n)
quad_cov_pump = sparse.bmat([[sparse.csc_matrix(((2*N_z + 1)*(N_omega**2),(2*N_z + 1)*(N_omega**2))), sparse.csc_matrix(((2*N_z + 1)*(N_omega**2), 2*N_omega - 1))],
             [sparse.csc_matrix((2*N_omega - 1, (2*N_z + 1)*(N_omega**2))), sparse.csc_matrix(np.diag(np.linspace(omega[0], omega[-1], 2*N_omega - 1)**2))]])
constr_cov_pump = sparse.bmat([[quad_cov_pump, sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,1))],
                               [sparse.csc_matrix((1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), -1.5*sparse.eye(1)]])
sdr_constr = sparse.bmat([[sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1, 1))],
                          [sparse.csc_matrix((1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), sparse.eye(1)]])
photon_nbr_constr_mat = vect.photon_numb_mat(N_omega, N_z, n)

In [58]:
# Fix values of beta
random_beta = beta_vec#np.random.random(2*N_omega - 1)
beta_def_constr_list_real = []
beta_def_constr_list_imag = []
for i in range(2*N_omega - 1):
    beta_proj = np.zeros(2*N_omega - 1)
    beta_proj[i] = 1
    beta_proj = sparse.csc_matrix(beta_proj.reshape((2*N_omega - 1, 1)))
    beta_proj = sparse.vstack([sparse.csc_matrix(((2*N_z + 1)*(N_omega**2),1)), beta_proj])
    beta_def_constr_list_real.append(sparse.bmat([[sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), 0.5*beta_proj],[0.5*beta_proj.conj().T, -random_beta[i]*sparse.eye(1)]]))
    beta_def_constr_list_imag.append(sparse.bmat([[sparse.csc_matrix(((2*N_z + 1)*(N_omega**2) + 2*N_omega - 1,(2*N_z + 1)*(N_omega**2) + 2*N_omega - 1)), 0.5*(beta_proj*1.j)],[0.5*(beta_proj*1.j).conj().T, sparse.csc_matrix((1,1))]]))

In [59]:
# Test the dynamics constraints matrices
# Make sure the value gives the same as when i do with explicit matmul of LS
delta_z = np.abs(z[1] - z[0])
test_green_fs = vect.get_green_f(omega, z)
test_green_fs.reverse()
test_green_fs[0] = test_green_fs[0]/2
test_green_fs[-1] = test_green_fs[-1]/2
true_dynamics_residue_plus = []
true_dynamics_residue_minus = []
test_dynamics_residue_plus = []
test_dynamics_residue_minus = []
for i in range(len(projections)):
    true_dynamics_residue_plus.append(-np.trace(projections[i].conj().T@W_plus[-1]) + 2*np.trace(projections[i].conj().T@test_green_fs[0]) + delta_z*np.trace(sum([projections[i].conj().T@test_green_fs[j]@beta@W_plus[j] for j in range(len(W_plus))])))
    true_dynamics_residue_minus.append(-np.trace(projections[i].conj().T@W_minus[-1]) + 2*np.trace(projections[i].conj().T@test_green_fs[0]) - delta_z*np.trace(sum([projections[i].conj().T@test_green_fs[j]@beta@W_minus[j] for j in range(len(W_plus))])))
    test_dynamics_residue_plus.append((y.conj().T@real_plus_mats_list[(i + 1)*N_z - 1]@y).trace() + 1.j*(y.conj().T@imag_plus_mats_list[(i + 1)*N_z - 1]@y).trace())
    test_dynamics_residue_minus.append((y.conj().T@real_minus_mats_list[(i + 1)*N_z - 1]@y).trace() + 1.j*(y.conj().T@imag_minus_mats_list[(i + 1)*N_z - 1]@y).trace())

In [60]:
np.allclose(test_dynamics_residue_plus, true_dynamics_residue_plus), np.allclose(test_dynamics_residue_minus, true_dynamics_residue_minus)

(True, True)

In [64]:
# using cvxpy
variable = cp.Variable(shape = ((2*N_z + 1)*(N_omega**2) + 2*N_omega, (2*N_z + 1)*(N_omega**2) + 2*N_omega), complex = True)
constraints = [variable >> 0]
constraints += [cp.real(cp.trace(real_plus_mats_list[i]@variable)) == 0 for i in range(len(real_plus_mats_list))]
constraints += [cp.real(cp.trace(imag_plus_mats_list[i]@variable)) == 0 for i in range(len(imag_plus_mats_list))]
constraints += [cp.real(cp.trace(real_minus_mats_list[i]@variable)) == 0 for i in range(len(real_minus_mats_list))]
constraints += [cp.real(cp.trace(imag_minus_mats_list[i]@variable)) == 0 for i in range(len(imag_minus_mats_list))]
constraints += [cp.real(cp.trace(J_def_mat[i]@variable)) == 0 for i in range(len(J_def_mat))]
constraints += [cp.real(cp.trace(real_symplectic_constr[i]@variable)) == 0 for i in range(len(real_symplectic_constr))]
constraints += [cp.real(cp.trace(imag_symplectic_constr[i]@variable)) == 0 for i in range(len(imag_symplectic_constr))]
constraints += [cp.real(cp.trace(beta_def_constr_list_real[i]@variable)) == 0 for i in range(2*N_omega - 1)]
constraints += [cp.real(cp.trace(beta_def_constr_list_imag[i]@variable)) == 0 for i in range(2*N_omega - 1)]
constraints.append(cp.real(cp.trace(photon_nbr_constr_mat@variable)) == 0)
constraints.append(cp.real(cp.trace(obj_f_mat@variable)) >= 0)
constraints.append(cp.real(cp.trace(constr_cov_pump@variable)) <= 0)
constraints.append(cp.real(cp.trace(sdr_constr@variable)) == 1.)
problem = cp.Problem(cp.Minimize(cp.real(cp.trace(obj_f_mat@variable))), constraints)

In [65]:
problem.solve(verbose = True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Feb 14 10:54:43 AM: Your problem has 616225 variables, 2298 constraints, and 0 parameters.
(CVXPY) Feb 14 10:54:44 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 14 10:54:44 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 14 10:54:44 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 14 10:54:44 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 14 10:54:44 AM: Compiling problem (target solver=SCS)